# 분류 - classification
    고객분류, 질병진단, 스펨메일 필터링, 음성인식등... 카테고리를 갖는 경우
     - KNN, SVM, Decision Tree, Logistic Regression     

### KNN K-Nearst-Neighbors

# 1. 데이터 준비(확보)

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [10]:
titanic = sns.load_dataset('titanic')
titanic.rename(columns={'sex':'gender'},inplace=True)

In [11]:
titanic.head()

,survived,pclass,gender,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


# 2.데이터 탐색 - info(), describe()

In [12]:
# 결측치 확인 - 1. 갯수, 2.형태는 숫자인데.. 타입이 object면 의심해 봐라.
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   gender       891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


In [13]:
# 결측치가 많은 컬럼은 통으로 제거하고. - deck 제거
# 의미가 비슷한 컬럼은 하나만 사용 -  embark_town 제거
titanic.drop(['deck','embark_town'],axis=1,inplace=True)

In [15]:
titanic.isna().sum()

survived        0
pclass          0
gender          0
age           177
sibsp           0
parch           0
fare            0
embarked        2
class           0
who             0
adult_male      0
alive           0
alone           0
dtype: int64

In [16]:
# embarked        2 --> 가장 많은 도시약자로 대체  fillna()

In [19]:
titanic['embarked'].value_counts()

S    644
C    168
Q     77
Name: embarked, dtype: int64

In [28]:
freq = titanic['embarked'].value_counts().idxmax()

In [29]:
titanic['embarked'].fillna(freq,inplace=True)

In [31]:
titanic.isnull().mean()

survived      0.000000
pclass        0.000000
gender        0.000000
age           0.198653
sibsp         0.000000
parch         0.000000
fare          0.000000
embarked      0.000000
class         0.000000
who           0.000000
adult_male    0.000000
alive         0.000000
alone         0.000000
dtype: float64

In [35]:
# age 처리여부. 
# 1 drop
# titanic = titanic.dropna(subset=['age'])

# 2.치환 - 선실등급(pclass)별 gender 별 평균으로 대체
grouped =  titanic.groupby(['pclass','gender'])

In [46]:
grouped_df = grouped['age'].mean()
# grouped_df[(1, 'female')]
grouped_df

pclass  gender
1       female    34.611765
        male      41.281386
2       female    28.722973
        male      30.740707
3       female    21.750000
        male      26.507589
Name: age, dtype: float64

In [53]:
# pclass == 1 gender == 'female'  34.611765
df = titanic[titanic.age.isnull()]

In [54]:
df.head()

,survived,pclass,gender,age,sibsp,parch,fare,embarked,class,who,adult_male,alive,alone
5,0,3,male,NaN,0,0,8.4583,Q,Third,man,True,no,True
17,1,2,male,NaN,0,0,13.0000,S,Second,man,True,yes,True
19,1,3,female,NaN,0,0,7.2250,C,Third,woman,False,yes,True
26,0,3,male,NaN,0,0,7.2250,C,Third,man,True,no,True
28,1,3,female,NaN,0,0,7.8792,Q,Third,woman,False,yes,True


In [61]:
idx1,idx2,idx3,idx4,idx5,idx6 = [],[],[],[],[],[]
for i in range(len(df)):
    if (df.iloc[i,1] == 1) & (df.iloc[i,2] == 'female') :        
        idx1.append(df.index[i])
    elif (df.iloc[i,1] == 1) & (df.iloc[i,2] == 'male') :
        idx2.append(df.index[i])
    elif (df.iloc[i,1] == 2) & (df.iloc[i,2] == 'female') :        
        idx3.append(df.index[i])
    elif (df.iloc[i,1] == 2) & (df.iloc[i,2] == 'male') :
        idx4.append(df.index[i])
    elif (df.iloc[i,1] == 3) & (df.iloc[i,2] == 'female') :        
        idx5.append(df.index[i])
    elif (df.iloc[i,1] == 4) & (df.iloc[i,2] == 'male') :
        idx6.append(df.index[i])    

In [74]:
# idx1 ~ idx5
titanic.iloc[idx1,3] = grouped_df.values[0]
titanic.iloc[idx2,3] = grouped_df.values[1]
titanic.iloc[idx3,3] = grouped_df.values[2]
titanic.iloc[idx4,3] = grouped_df.values[3]
titanic.iloc[idx5,3] = grouped_df.values[4]

In [77]:
titatitanic.age.isnull()

0      False
1      False
2      False
3      False
4      False
       ...  
886    False
887    False
888    False
889    False
890    False
Name: age, Length: 891, dtype: bool